In [2]:
%load_ext autoreload
%autoreload 2

import os
import sys
import time

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from data_manager import *
from metadata import *
from distributed_utilities import *

Setting environment for AWS compute node
rm -rf /shared/CSHL_data_processed/MD585/MD585_thumbnail_alignedTo_MD585-N47-2015.07.16-22.50.52_MD585_3_0141_cropped/MD585-IHC18-2015.07.16-15.35.58_MD585_1_0052_thumbnail_alignedTo_MD585-N47-2015.07.16-22.50.52_MD585_3_0141_cropped.tif && mkdir -p /shared/CSHL_data_processed/MD585/MD585_thumbnail_alignedTo_MD585-N47-2015.07.16-22.50.52_MD585_3_0141_cropped
aws s3 cp s3://mousebrainatlas-data/CSHL_data_processed/MD585/MD585_thumbnail_alignedTo_MD585-N47-2015.07.16-22.50.52_MD585_3_0141_cropped/MD585-IHC18-2015.07.16-15.35.58_MD585_1_0052_thumbnail_alignedTo_MD585-N47-2015.07.16-22.50.52_MD585_3_0141_cropped.tif /shared/CSHL_data_processed/MD585/MD585_thumbnail_alignedTo_MD585-N47-2015.07.16-22.50.52_MD585_3_0141_cropped/MD585-IHC18-2015.07.16-15.35.58_MD585_1_0052_thumbnail_alignedTo_MD585-N47-2015.07.16-22.50.52_MD585_3_0141_cropped.tif


No vtk
Child returned 0
Child returned 0
0.47 seconds.
Child returned 0


rm -rf /shared/CSHL_data_processed/MD589/MD589_thumbnail_alignedTo_MD589-IHC31-2015.07.30-23.26.22_MD589_1_0091_cropped/MD589-IHC68-2015.07.31-02.03.18_MD589_1_0202_thumbnail_alignedTo_MD589-IHC31-2015.07.30-23.26.22_MD589_1_0091_cropped.tif && mkdir -p /shared/CSHL_data_processed/MD589/MD589_thumbnail_alignedTo_MD589-IHC31-2015.07.30-23.26.22_MD589_1_0091_cropped
aws s3 cp s3://mousebrainatlas-data/CSHL_data_processed/MD589/MD589_thumbnail_alignedTo_MD589-IHC31-2015.07.30-23.26.22_MD589_1_0091_cropped/MD589-IHC68-2015.07.31-02.03.18_MD589_1_0202_thumbnail_alignedTo_MD589-IHC31-2015.07.30-23.26.22_MD589_1_0091_cropped.tif /shared/CSHL_data_processed/MD589/MD589_thumbnail_alignedTo_MD589-IHC31-2015.07.30-23.26.22_MD589_1_0091_cropped/MD589-IHC68-2015.07.31-02.03.18_MD589_1_0202_thumbnail_alignedTo_MD589-IHC31-2015.07.30-23.26.22_MD589_1_0091_cropped.tif


Child returned 0
0.47 seconds.
Child returned 0


rm -rf /shared/CSHL_data_processed/MD590/MD590_thumbnail_alignedTo_MD590-N47-2015.09.12-05.32.06_MD590_2_0140_cropped/MD590-IHC17-2015.08.10-19.09.09_MD590_2_0050_thumbnail_alignedTo_MD590-N47-2015.09.12-05.32.06_MD590_2_0140_cropped.tif && mkdir -p /shared/CSHL_data_processed/MD590/MD590_thumbnail_alignedTo_MD590-N47-2015.09.12-05.32.06_MD590_2_0140_cropped
aws s3 cp s3://mousebrainatlas-data/CSHL_data_processed/MD590/MD590_thumbnail_alignedTo_MD590-N47-2015.09.12-05.32.06_MD590_2_0140_cropped/MD590-IHC17-2015.08.10-19.09.09_MD590_2_0050_thumbnail_alignedTo_MD590-N47-2015.09.12-05.32.06_MD590_2_0140_cropped.tif /shared/CSHL_data_processed/MD590/MD590_thumbnail_alignedTo_MD590-N47-2015.09.12-05.32.06_MD590_2_0140_cropped/MD590-IHC17-2015.08.10-19.09.09_MD590_2_0050_thumbnail_alignedTo_MD590-N47-2015.09.12-05.32.06_MD590_2_0140_cropped.tif


...

Child returned 0
0.47 seconds.
Child returned 0


rm -rf /shared/CSHL_data_processed/MD642/MD642_thumbnail_alignedTo_MD642-F53-2017.01.14-12.23.43_MD642_1_0157_cropped/MD642-F40-2017.01.14-08.03.13_MD642_1_0118_thumbnail_alignedTo_MD642-F53-2017.01.14-12.23.43_MD642_1_0157_cropped.tif && mkdir -p /shared/CSHL_data_processed/MD642/MD642_thumbnail_alignedTo_MD642-F53-2017.01.14-12.23.43_MD642_1_0157_cropped
aws s3 cp s3://mousebrainatlas-data/CSHL_data_processed/MD642/MD642_thumbnail_alignedTo_MD642-F53-2017.01.14-12.23.43_MD642_1_0157_cropped/MD642-F40-2017.01.14-08.03.13_MD642_1_0118_thumbnail_alignedTo_MD642-F53-2017.01.14-12.23.43_MD642_1_0157_cropped.tif /shared/CSHL_data_processed/MD642/MD642_thumbnail_alignedTo_MD642-F53-2017.01.14-12.23.43_MD642_1_0157_cropped/MD642-F40-2017.01.14-08.03.13_MD642_1_0118_thumbnail_alignedTo_MD642-F53-2017.01.14-12.23.43_MD642_1_0157_cropped.tif


Child returned 0
0.46 seconds.
Child returned 0


rm -rf /shared/CSHL_data_processed/MD658/MD658_thumbnail_alignedTo_MD658-N58-2017.03.31-19.59.31_MD658_2_0173_cropped/MD658-F73-2017.04.05-10.27.52_MD658_3_0219_thumbnail_alignedTo_MD658-N58-2017.03.31-19.59.31_MD658_2_0173_cropped.tif && mkdir -p /shared/CSHL_data_processed/MD658/MD658_thumbnail_alignedTo_MD658-N58-2017.03.31-19.59.31_MD658_2_0173_cropped
aws s3 cp s3://mousebrainatlas-data/CSHL_data_processed/MD658/MD658_thumbnail_alignedTo_MD658-N58-2017.03.31-19.59.31_MD658_2_0173_cropped/MD658-F73-2017.04.05-10.27.52_MD658_3_0219_thumbnail_alignedTo_MD658-N58-2017.03.31-19.59.31_MD658_2_0173_cropped.tif /shared/CSHL_data_processed/MD658/MD658_thumbnail_alignedTo_MD658-N58-2017.03.31-19.59.31_MD658_2_0173_cropped/MD658-F73-2017.04.05-10.27.52_MD658_3_0219_thumbnail_alignedTo_MD658-N58-2017.03.31-19.59.31_MD658_2_0173_cropped.tif


Child returned 0
0.52 seconds.


# Compute scoremaps for linearly normalized version

In [18]:
stack = 'MD657'

for classifier_id in range(83, 85):
    t = time.time()
    sys.stderr.write('running svm classifier ...')

    run_distributed(command='%(script_path)s %(stack)s \'%%(filenames)s\' %(classifier_id)d' % \
                    {'script_path': os.path.join(REPO_DIR, 'learning', 'apply_classifiers_v4.py'),
                    'stack': stack,
                    'classifier_id': classifier_id},
    #                 kwargs_list=dict(filenames=metadata_cache['valid_filenames'][stack]),
    #                 kwargs_list=dict(filenames=[metadata_cache['sections_to_filenames'][stack][150]]),
                    kwargs_list=dict(filenames=[fn for fn in metadata_cache['valid_filenames'][stack]
                                               if fn.split('-')[1][0] == 'F']),
                    argument_type='list2')

    wait_qsub_complete()

    sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 302s

    t = time.time()
    sys.stderr.write('Resampling scoremaps ...')

    # downscale = 8
    downscale = 32

    run_distributed(command='%(script_path)s %(stack)s \'%%(filenames)s\' %(classifier_id)d %(downscale)d' % \
                    {'script_path': os.path.join(REPO_DIR, 'learning', 'resample_scoremaps_v4.py'),
                    'stack': stack,
                    'classifier_id': classifier_id,
                    'downscale': downscale},
    #                 kwargs_list=dict(filenames=metadata_cache['valid_filenames'][stack]),
    #                 kwargs_list=dict(filenames=[metadata_cache['sections_to_filenames'][stack][150]]),
                    kwargs_list=dict(filenames=[fn for fn in metadata_cache['valid_filenames'][stack]
                               if fn.split('-')[1][0] == 'F']),
                    argument_type='list2'
                   )


    wait_qsub_complete()

    sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 277s

running svm classifier ...Child returned 0
16 nodes available.


rm -f ~/stderr_*; rm -f ~/stdout_*


Jobs submitted. Use wait_qsub_complete() to wait for all execution to finish.
qsub returned.
done in 317.484612 seconds
Resampling scoremaps ...Child returned 0
16 nodes available.


rm -f ~/stderr_*; rm -f ~/stdout_*


Jobs submitted. Use wait_qsub_complete() to wait for all execution to finish.
qsub returned.
done in 70.839120 seconds
running svm classifier ...Child returned 0
16 nodes available.


rm -f ~/stderr_*; rm -f ~/stdout_*


Jobs submitted. Use wait_qsub_complete() to wait for all execution to finish.
qsub returned.
done in 322.478262 seconds
Resampling scoremaps ...Child returned 0
16 nodes available.


rm -f ~/stderr_*; rm -f ~/stdout_*


Jobs submitted. Use wait_qsub_complete() to wait for all execution to finish.
qsub returned.
done in 65.922713 seconds


# Regular version

In [52]:
classifier_id = 38

In [53]:
stack = 'MD590'

In [13]:
t = time.time()
sys.stderr.write('running svm classifier ...')

run_distributed(command='%(script_path)s %(stack)s \'%%(filenames)s\' %(classifier_id)d' % \
                {'script_path': os.path.join(REPO_DIR, 'learning', 'apply_classifiers_v4.py'),
                'stack': stack,
                'classifier_id': classifier_id},
                kwargs_list=dict(filenames=metadata_cache['valid_filenames'][stack]),
#                 kwargs_list=dict(filenames=[metadata_cache['sections_to_filenames'][stack][208]]),
#                 kwargs_list=dict(filenames=[fn for fn in metadata_cache['valid_filenames'][stack]
#                                            if fn.split('-')[1][0] == 'F']),
                argument_type='list2')

wait_qsub_complete()

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 302s

running svm classifier ...Child returned 0
8 nodes available.
Jobs submitted. Use wait_qsub_complete() to wait for all execution to finish.


rm -f ~/stderr_*; rm -f ~/stdout_*


qsub returned.
done in 295.906637 seconds


In [84]:
# Feature computation window spacing is 56, so the equivalent downscale factor is 56. Still larger than 32.

In [54]:
t = time.time()
sys.stderr.write('Resampling scoremaps ...')

downsample = 8
# downsample = 32

run_distributed(command='%(script_path)s %(stack)s \'%%(filenames)s\' %(classifier_id)d %(downsample)d' % \
                {'script_path': os.path.join(REPO_DIR, 'learning', 'resample_scoremaps_v4.py'),
                'stack': stack,
                'classifier_id': classifier_id,
                'downsample': downsample},
                kwargs_list=dict(filenames=metadata_cache['valid_filenames'][stack]),
#                 kwargs_list=dict(filenames=[metadata_cache['sections_to_filenames'][stack][209]]),
#                 kwargs_list=dict(filenames=[fn for fn in metadata_cache['valid_filenames'][stack]
#                            if fn.split('-')[1][0] == 'F']),
                argument_type='list2'
               )


wait_qsub_complete()

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 277s

Resampling scoremaps ...8 nodes available.
Jobs submitted. Use wait_qsub_complete() to wait for all execution to finish.


rm -f ~/stderr_*; rm -f ~/stdout_*


qsub returned.
done in 135.525693 seconds


In [59]:
t = time.time()
sys.stderr.write('visualize scoremaps ...')

add_label_text = False
viz_downsample = 8
# viz_downsample = 32
cmap = 'jet'

run_distributed(command='%(script_path)s %(stack)s \'%%(filenames)s\' %(classifier_id)d %(downsample)d --cmap %(cmap)s %(add_label_text)s' % \
                {'script_path': os.path.join(REPO_DIR, 'learning', 'visualize_scoremaps_v4.py'),
                'stack': stack,
                 'classifier_id': classifier_id,
                'downsample': viz_downsample,
                'add_label_text': '-a' if add_label_text else ''},
                kwargs_list=dict(filenames=metadata_cache['valid_filenames'][stack]),
#                 kwargs_list=dict(filenames=[metadata_cache['sections_to_filenames'][stack][209]]),
                argument_type='list2'
               )

wait_qsub_complete()

sys.stderr.write('done in %f seconds\n' % (time.time() - t))  
# For one stack
# 16 nodes, downsample 32, 75s
# 8 nodes, downscale 8, 6000s

visualize scoremaps ...8 nodes available.
Jobs submitted. Use wait_qsub_complete() to wait for all execution to finish.


rm -f ~/stderr_*; rm -f ~/stdout_*


qsub returned.
done in 5893.626925 seconds


In [23]:
t = time.time()
sys.stderr.write('constructing score volumes ...')

use_nissl_only = False
volume_downscale = 32

run_distributed(command='%(script_path)s %(stack)s %%(structure)s %(classifier_id)d %(downscale)d %(use_nissl_only)s' % \
                {'script_path': os.path.join(REPO_DIR, 'reconstruct', 'construct_score_volume_v4.py'),
                'stack': stack,
                'classifier_id': classifier_id,
                'downscale': volume_downscale,
                'use_nissl_only': '-n' if use_nissl_only else ''},
#                 kwargs_list=dict(structure=all_known_structures),
                kwargs_list=dict(structure=['VLL']),
                argument_type='single')

wait_qsub_complete()

sys.stderr.write('done in %f seconds\n' % (time.time() - t))# 116s

constructing score volumes ...Child returned 0
8 nodes available.
Jobs submitted. Use wait_qsub_complete() to wait for all execution to finish.


rm -f ~/stderr_*; rm -f ~/stdout_*


qsub returned.
done in 20.199607 seconds


In [25]:
t = time.time()
sys.stderr.write('compute score volume gradients...')

run_distributed(command='ROOT_DIR=/scratch/ %(script_path)s %(stack)s %%(structure)s %(classifier_id)d %(downscale)d' % \
                {'script_path': os.path.join(REPO_DIR, 'reconstruct', 'compute_score_volume_gradients_v4.py'),
                'stack': stack,
                'classifier_id': classifier_id,
                'downscale': volume_downscale},
                kwargs_list=dict(structure=all_known_structures),
#                 kwargs_list=dict(structure=['VLL']),
                argument_type='single')

wait_qsub_complete()

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) 

# 55 seconds (16 nodes write to respective local /scratch)
# More than 1 simul. processes are not beneficial as they cause local write contention.
# 156 seconds (16 nodes simul. write to /shared)
# 310 seconds (single node, sequential write to /shared)

compute score volume gradients...Child returned 0
8 nodes available.


rm -f ~/stderr_*; rm -f ~/stdout_*


Jobs submitted. Use wait_qsub_complete() to wait for all execution to finish.
qsub returned.
done in 120.656658 seconds


# Global alignment

In [21]:
classifier_setting = 38
warp_setting = 13

In [22]:
# structure_subset = ['7N_L', '7N_R', '12N', '5N_L','5N_R','Pn_L', 'Pn_R', 'SNR_L', 'SNR_R', 'VLL_L', 'VLL_R', '7n_L']
# structure_subset = ['7N_L', '7N_R', '12N', '5N_L','5N_R','Pn_L', 'Pn_R', 'SNR_L', 'SNR_R', 
#                     'VLL_L', 'VLL_R', '7n_L', '7n_R', 'Tz_L', 'Tz_R', 
#                     'VCA_L', 'VCP_R']
# structure_subset = ['7N_L', '7N_R', '12N', '5N_L','5N_R','Pn_L', 'Pn_R', 'SNR_L', 'SNR_R', 
#                     'VLL_L', 'VLL_R', '7n_L', '7n_R', 'Tz_L', 'Tz_R', 
#                     'VCA_L', 'VCP_R', 'IC', 'SC']
structure_subset = ['7N_L', '7N_R', '12N', '5N_L', '5N_R', 'SNR_L', 'SNR_R', 'Pn_L', 'Pn_R',
                    'VLL_L', 'VLL_R', '7n_L', '7n_R', 'Tz_L', 'Tz_R', 
                    'VCA_L', 'VCA_R', 'VCP_L', 'VCP_R']

In [23]:
atlas_name = 'atlasV4'

In [24]:
# Align

t = time.time()
sys.stderr.write('align all subjects to atlas ...')

run_distributed(command='%(script_path)s %%(stack)s %(atlas_name)s %(warp_setting)d %(clf_setting)d --trial_num 5 --structures \'%(structures)s\'' % \
                {'script_path': os.path.join(REPO_DIR, 'registration', 'global_registration_v3.py'),
                 'atlas_name': atlas_name,
                'warp_setting': warp_setting,
                'clf_setting': classifier_setting,
                'structures': json.dumps(structure_subset)},
                 kwargs_list=dict(stack=['MD635']),
                argument_type='single')

wait_qsub_complete()

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # For one stack, 372s (1 trial); 750s (5 trials)

align all subjects to atlas ...Child returned 0
1 nodes available.
Jobs submitted. Use wait_qsub_complete() to wait for all execution to finish.


rm -f ~/stderr_*; rm -f ~/stdout_*


qsub returned.
done in 2728.219750 seconds


In [25]:
# Transform

t = time.time()
sys.stderr.write('transform atlas ...')

run_distributed(command='%(script_path)s %%(stack)s %(atlas_name)s %(warp_setting)d %(clf_setting)d' % \
                {'script_path': os.path.join(REPO_DIR, 'registration', 'transform_brains_v4.py'),
                'atlas_name': atlas_name,
                'clf_setting': classifier_setting,
                 'warp_setting': warp_setting
                },
                kwargs_list={'stack': ['MD635']},
                argument_type='single')

wait_qsub_complete()

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # For one stack, 60 seconds

transform atlas ...Child returned 0
1 nodes available.
Jobs submitted. Use wait_qsub_complete() to wait for all execution to finish.


rm -f ~/stderr_*; rm -f ~/stdout_*


qsub returned.
done in 45.220334 seconds


In [26]:
# Visualize

t = time.time()
sys.stderr.write('visualize aligned atlas overlay ...')

run_distributed(command='%(script_path)s %%(stack)s %(atlas_name)s %(warp_setting)d %(clf_setting)d --out_downsample 8' % \
                {'script_path': os.path.join(REPO_DIR, 'registration', 'visualize_registration_v4.py'),
                'atlas_name': atlas_name,
                'clf_setting': classifier_setting,
                 'warp_setting': warp_setting
                },
                kwargs_list={'stack': ['MD635']},
                argument_type='single')

wait_qsub_complete()

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # For one stack, 110 seconds

visualize aligned atlas overlay ...Child returned 0
1 nodes available.
Jobs submitted. Use wait_qsub_complete() to wait for all execution to finish.


rm -f ~/stderr_*; rm -f ~/stdout_*


qsub returned.
done in 85.274688 seconds


# Local align

In [56]:
classifier_setting = 38
warp_setting = 9

In [57]:
atlas_name = 'atlasV4'

In [36]:
# Local align

t = time.time()
sys.stderr.write('LOCAL align ...')

kwargs_list = \
[dict(stack='MD635', structures=shell_escape(json.dumps(all_known_structures_sided[f:l+1])))
for f,l in first_last_tuples_distribute_over(0, len(all_known_structures_sided)-1, get_num_nodes())]

run_distributed(command='%(script_path)s %%(stack)s %(atlas_name)s %(warp_setting)d %(clf_setting)d -s %%(structures)s' % \
                {'script_path': os.path.join(REPO_DIR, 'registration', 'local_registration_v3.py'),
                'clf_setting': classifier_setting,
                 'warp_setting': warp_setting,
                'atlas_name': atlas_name},
                 kwargs_list=kwargs_list,
                argument_type='single')

wait_qsub_complete()

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 2047s for 5 trials.
# IC and SC are super slow...

LOCAL align ...8 nodes available.


rm -f ~/stderr_*; rm -f ~/stdout_*


Jobs submitted. Use wait_qsub_complete() to wait for all execution to finish.


Exception: qsub does not return in 7200 seconds. Quit waiting, but SGE may still be computing..

In [45]:
# Transform

t = time.time()
sys.stderr.write('transform atlas ...')

run_distributed(command='%(script_path)s %%(stack)s %(atlas_name)s %(warp_setting)d %(clf_setting)d' % \
                {'script_path': os.path.join(REPO_DIR, 'registration', 'transform_brains_v4.py'),
                'clf_setting': classifier_setting,
                 'warp_setting': warp_setting,
                'atlas_name': atlas_name
                },
                kwargs_list={'stack': ['MD635']},
                argument_type='single')

wait_qsub_complete()

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # For one stack, 60 seconds

transform atlas ...8 nodes available.
Jobs submitted. Use wait_qsub_complete() to wait for all execution to finish.


rm -f ~/stderr_*; rm -f ~/stdout_*


qsub returned.
done in 45.241995 seconds


In [58]:
# Visualize

t = time.time()
sys.stderr.write('visualize aligned atlas overlay ...')

run_distributed(command='%(script_path)s %%(stack)s %(atlas_name)s %(warp_setting)d %(clf_setting)d --out_downsample 8' % \
                {'script_path': os.path.join(REPO_DIR, 'registration', 'visualize_registration_v4.py'),
                'clf_setting': classifier_setting,
                 'warp_setting': warp_setting,
                'atlas_name': atlas_name
                },
                kwargs_list={'stack': ['MD635']},
                argument_type='single')

wait_qsub_complete()

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # For one stack, 110 seconds

visualize aligned atlas overlay ...8 nodes available.
Jobs submitted. Use wait_qsub_complete() to wait for all execution to finish.


rm -f ~/stderr_*; rm -f ~/stdout_*


qsub returned.
done in 182.591091 seconds


# Transform locally transformed volumes back to atlas space

In [ ]:
classifier_setting = 2
warp_setting = 3

In [ ]:
t = time.time()
sys.stderr.write('transform atlas ...')

exclude_nodes = [33]

run_distributed4(command='%(script_path)s %%(stack)s atlasV2 %(warp_setting)d %(clf_setting)d' % \
                {'script_path': os.path.join(os.environ['REPO_DIR'], 'registration', 'transform_brains_v3_reverse_global.py'),
                 'clf_setting': classifier_setting,
                 'warp_setting': warp_setting},
                 kwargs_list=dict(stack=['MD603']),
#                 kwargs_list=dict(stack=all_nissl_stacks),
                exclude_nodes=exclude_nodes,
                argument_type='single')

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 200 seconds ~ 4 mins

# Measure Global Tx Confidence

In [ ]:
warp_setting = 1

In [ ]:
t = time.time()
sys.stderr.write('Measure confidence ...')

exclude_nodes = [33]

run_distributed4(command='%(script_path)s %%(stack)s atlasV2 %(warp_setting)d %(clf_setting)d' % \
                {'script_path': os.path.join(os.environ['REPO_DIR'], 'registration', 'measure_confidence_v3_global.py'),
                 'clf_setting': classifier_setting,
                 'warp_setting': warp_setting},
#                  kwargs_list=dict(stack=['MD594']),
                kwargs_list=dict(stack=all_nissl_stacks),
                exclude_nodes=exclude_nodes,
                argument_type='single')

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 650 seconds ~ 10 mins

# Measure Local Tx Confidence

In [ ]:
warp_setting = 5
classifier_setting = 2

In [ ]:
t = time.time()
sys.stderr.write('Measure confidence ...')

exclude_nodes = [33]

run_distributed4(command='%(script_path)s %%(stack)s atlasV2 %(warp_setting)d %(clf_setting)d' % \
                {'script_path': os.path.join(os.environ['REPO_DIR'], 'registration', 'measure_confidence_v3_local.py'),
                 'clf_setting': classifier_setting,
                 'warp_setting': warp_setting},
#                  kwargs_list=dict(stack=['MD592']),
                kwargs_list=dict(stack=all_nissl_stacks),
                exclude_nodes=exclude_nodes,
                argument_type='single')

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 900 s ~ 15 mins